In [1]:
import sys
sys.path.append('../../')

In [2]:
import actionlib
import Config
sys.path.append(Config.MISC_DIR+"real_world_exec/")
from fetch.fetch_control import Gripper, Head
from control_msgs.msg import FollowJointTrajectoryGoal, FollowJointTrajectoryAction
import rospy
import pickle
import actionlib
import rospy
import sys, time
import math
from openravepy import *
from smach import State,StateMachine
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal

In [3]:
with open(Config.DOMAIN_DIR +'ros_trajectory.pickle', 'r') as f:
    trajectories = pickle.load(f)
    f.close()

In [4]:
rospy.init_node("physical_fetch_execution")

In [5]:
def execute_manip_traj_on_ros(ros_manip_traj):
    client = actionlib.SimpleActionClient('/arm_with_torso_controller/follow_joint_trajectory',
                                          FollowJointTrajectoryAction)
    client.wait_for_server()
    id_gen = actionlib.GoalIDGenerator()
    jtg = FollowJointTrajectoryGoal()
    jtg.trajectory = ros_manip_traj
    client.send_goal(jtg)
    client.wait_for_result()   

In [6]:
class NavWaypoint(State):
    def __init__(self, goal):
        State.__init__(self, outcomes=['success'])
        # Get an action client
        self.client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
        self.client.wait_for_server()
        # Define the goal
        self.goal = goal

    def execute(self, userdata):
        self.client.send_goal(self.goal)
        self.client.wait_for_result()
        return 'success'
def execute_base_traj_on_ros(ros_base_traj):
    patrol = StateMachine('success')
    num_waypts = len(ros_base_traj)
    with patrol:
        for i,waypt in enumerate(ros_base_traj):
            StateMachine.add(str(i), NavWaypoint(waypt),transitions={'success':str((i + 1) % num_waypts)})
    patrol.execute()

In [7]:
gripper = Gripper()

In [9]:
for i in range(len(trajectories)):
    t_type, t_val = trajectories[i]
    if t_type == 'GripperOpenTrajectory':
        gripper.open()
    elif t_type == 'GripperCloseTrajectory':
        gripper.close()
    elif t_type == 'ManipTrajectory':
        execute_manip_traj_on_ros(t_val)
    elif t_type == 'BaseTrajectory':
        execute_base_traj_on_ros(t_val)